In [1]:
import reg_mri
import os
from glob import glob
from utils import compute_mean_dice
import nibabel as nib
from scipy.spatial.distance import dice
import numpy as np
import itk
import SimpleITK as sitk
import scipy.ndimage
import scipy
import matplotlib.pyplot as plt
from transforms_dict import getRegistrationEvalInverseTransformForMRI, SaveTransformForMRI
from tqdm import tqdm
import monai
import subprocess

In [2]:
def getLog10Jacobian_deformable_ants(warp_img, outfolder, outdataset):
    outdir = os.path.join('output', outdataset, outfolder, 'Log10Jacobian_Deformable')
    outname = outdir + '/Log10JacobianDeformable_' + warp_img.split('/')[-1].split('Warp')[0] + '.nii.gz'
    print(outname)
    jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warp_img) + " " + str(outname) + " 1 1"
    subprocess.call(jacobian_command.split(" "))

In [7]:
models = [
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch25.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch50.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch75.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch100.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch125.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch150.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch175.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch200.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch225.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch250.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch275.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch300.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch325.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch350.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch375.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch400.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch425.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch450.pth",
    "smoothloss_ct2_template_local_scenario1_feminad_newmodel_1.0-0.0-8.0_epoch475.pth",
         ]

for i, model in enumerate(models):
    outfolder = "testall_smoothloss"
    outdataset = "Feminad"
    mris = sorted(glob(os.path.join('dataset2', 'Feminad', 'MRI_N4_Resample_Norm_Identity_Affine', "*.nii.gz")))
    atlas_name = "dataset2/Atlas/Identity_Feminad_Template.nii.gz"
    affine = nib.load(atlas_name).affine
    header = nib.load(atlas_name).header 

    for i, mri in enumerate(mris):
        modelshort = model.split('.pth')[0].split('_epoch')[-1]
        outname = "output/" + outdataset + "/" + outfolder + "/MRI_N4_Registration_Deformable/" + mri.split('/')[-1].split('.')[0] + '_epoch' + modelshort
        pred_image, ddfs = reg_mri.main(model, mri, outname, False, True, "local", newmodel=True)
    
        moving_image = ddfs[0].cpu() 
    
        deformable_ddf = ddfs[1].cpu()
        deformable_ddf = deformable_ddf.cpu().numpy()
        deformable_ddf = np.transpose(deformable_ddf, (2, 3, 4, 0, 1))
    
        deformable_image = nib.Nifti1Image(deformable_ddf, affine, header)
        outname_deformable = "output/Feminad/testall_smoothloss/DeformableWarp/" + mri.split('/')[-1].split('.')[0] + '_epoch' + modelshort + ".nii.gz"
        nib.save(deformable_image, outname_deformable)
        break
        
mridir = 'output/Feminad/testall_smoothloss'
deformable_warps = sorted(glob(os.path.join(mridir, 'DeformableWarp', "*.nii.gz")))
for i in range(len(deformable_warps)):
    print(deformable_warps[i])
    getLog10Jacobian_deformable_ants(deformable_warps[i], outfolder, outdataset)  

=> Saved to output/Feminad/testall_smoothloss/MRI_N4_Registration_Deformable/Affine_Identity_Norm_Resampled_N4_12_6517_epoch25_MRI.nii.gz
=> Saved to output/Feminad/testall_smoothloss/MRI_N4_Registration_Deformable/Affine_Identity_Norm_Resampled_N4_12_6517_epoch25_Resample.nii.gz
=> Saved to output/Feminad/testall_smoothloss/MRI_N4_Registration_Deformable/Affine_Identity_Norm_Resampled_N4_12_6517_epoch25_Resample_Reg.nii.gz
=> Saved to output/Feminad/testall_smoothloss/MRI_N4_Registration_Deformable/Affine_Identity_Norm_Resampled_N4_12_6517_epoch25_Reg.nii.gz
1.7798106270000034
=> Saved to output/Feminad/testall_smoothloss/MRI_N4_Registration_Deformable/Affine_Identity_Norm_Resampled_N4_12_6517_epoch50_MRI.nii.gz
=> Saved to output/Feminad/testall_smoothloss/MRI_N4_Registration_Deformable/Affine_Identity_Norm_Resampled_N4_12_6517_epoch50_Resample.nii.gz
=> Saved to output/Feminad/testall_smoothloss/MRI_N4_Registration_Deformable/Affine_Identity_Norm_Resampled_N4_12_6517_epoch50_Resampl

=> Saved to output/Feminad/testall_smoothloss/MRI_N4_Registration_Deformable/Affine_Identity_Norm_Resampled_N4_12_6517_epoch375_MRI.nii.gz
=> Saved to output/Feminad/testall_smoothloss/MRI_N4_Registration_Deformable/Affine_Identity_Norm_Resampled_N4_12_6517_epoch375_Resample.nii.gz
=> Saved to output/Feminad/testall_smoothloss/MRI_N4_Registration_Deformable/Affine_Identity_Norm_Resampled_N4_12_6517_epoch375_Resample_Reg.nii.gz
=> Saved to output/Feminad/testall_smoothloss/MRI_N4_Registration_Deformable/Affine_Identity_Norm_Resampled_N4_12_6517_epoch375_Reg.nii.gz
1.7145427449999886
=> Saved to output/Feminad/testall_smoothloss/MRI_N4_Registration_Deformable/Affine_Identity_Norm_Resampled_N4_12_6517_epoch400_MRI.nii.gz
=> Saved to output/Feminad/testall_smoothloss/MRI_N4_Registration_Deformable/Affine_Identity_Norm_Resampled_N4_12_6517_epoch400_Resample.nii.gz
=> Saved to output/Feminad/testall_smoothloss/MRI_N4_Registration_Deformable/Affine_Identity_Norm_Resampled_N4_12_6517_epoch400_